In [450]:
import numpy as np
import pandas as pd
import seaborn as sns
import spacy as sp
import matplotlib.pyplot as plt
import nltk as nl
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from spacy.lang.en.stop_words import STOP_WORDS
from langdetect import detect

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.metrics import f1_score, confusion_matrix



In [481]:
X = pd.read_csv("train.csv")
XX = pd.read_csv("test.csv")

In [482]:
data_train = X.copy()
data_test = XX.copy()
print(X.describe())
X.info()

                 id  retweet_count  sentiment_class
count  3.235000e+03    3235.000000      3235.000000
mean   1.246352e+18       0.708501        -0.001236
std    4.814373e+15       4.127832         0.688719
min    1.240000e+18       0.000000        -1.000000
25%    1.240000e+18       0.000000         0.000000
50%    1.250000e+18       0.000000         0.000000
75%    1.250000e+18       0.000000         0.000000
max    1.250000e+18     118.000000         1.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3235 entries, 0 to 3234
Data columns (total 6 columns):
id                 3235 non-null float64
original_text      3235 non-null object
lang               3231 non-null object
retweet_count      3235 non-null int64
original_author    3235 non-null object
sentiment_class    3235 non-null int64
dtypes: float64(1), int64(2), object(3)
memory usage: 151.8+ KB


In [488]:
len(XX)

1387

In [211]:
# for i in range(len(lang)):
#     if(lang[i]!='en'):
#         print(i, lang[i])

185 de
683 ar
974 fr
1022 sl
1238 ja
1301 ar
1328 ar
1530 so
1866 tl
1943 ar
2207 ar
2400 ar
2647 ar
2650 it
2984 de
3006 ar


In [6]:
data_train.lang = [detect(wd) for wd in data_train.original_text]
data_test.lang = [detect(wd1) for wd1 in data_test.original_text]

In [212]:
# lang1 = [detect(wd) for wd in data_test.original_text]


In [ ]:
# print( [(i, data_train.lang[i]) for i in range(len(data_train.lang)) if data_train.lang[i] != 'en'])

In [7]:
print([data_train.drop(i, axis = 0, inplace=True) for i in range(len(data_train.lang)) if data_train.lang[i] != 'en'])
print([data_test.drop(i, axis = 0, inplace=True) for i in range(len(data_test.id)) if data_test.lang[i] != 'en'])


[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
[None, None, None, None, None, None, None, None]


In [243]:
nlp = sp.load("en_core_web_sm")
nl.download('words')
words = set(nl.corpus.words.words())

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Kaush\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [245]:
def processing(txt):
    txt = txt.lower()
    txt_l = re.sub('(\â)|(\€™)|(€)','',txt)
    txt_l = re.sub("(#)[a-zA-Z0-9]", " ", txt_l)
    txt_l = re.sub(r"(twitter.com).*?[a-zA-Z0-9].*\s", " ", txt_l)
    txt_l = re.sub(r"(https://).*(.com).*/([a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))", " ", txt_l)
    txt_l = " ".join(w for w in nl.wordpunct_tokenize(txt_l) if w.lower() in words or not w.isalpha())
    txt_ = nlp(txt_l)
    txt1 = [wd.text for wd in txt_ if wd.is_stop != True and wd.pos_ != 'PUNCT' ]
    txt1 = " ".join(txt1)
    txt1 = re.sub("[$-_@.&+]", "", txt1)
    txt1 = re.sub("\s+", " ", txt1)
    
    return(txt1)

In [246]:
data_train.original_text = data_train.original_text.apply(processing)


In [423]:
print(type(data_train.original_text[0]))

<class 'str'>


In [430]:
data = pd.DataFrame(columns = ['idx', 'original_text'])
data['idx'] = data_train.id.copy()
data['original_text'] = data_train.original_text.copy()
print(data)

X_train, X_test, y_train, y_test = train_test_split(data, data_train.sentiment_class, test_size = 0.2, random_state=10)

               idx                                      original_text
0     1.250000e+18  happy amazing know s hard able today s protect...
1     1.250000e+18  happy day mum m sorry t bring day honestly poi...
2     1.250000e+18  happy day days work today quiet time reflect d...
3     1.240000e+18  happy day beautiful woman royalty mummy emeral...
4     1.240000e+18   amazing ladies late grandmother iris mum caro...
...            ...                                                ...
3230  1.250000e+18  law wishing happy day praising day glory lord ...
3231  1.240000e+18  happy mother s day step amazing othersday cwhu...
3232  1.250000e+18  happy day woman know thanks pushing best perso...
3233  1.240000e+18  happy mother s day amazing wife love like craz...
3234  1.250000e+18  wishing safe happy day ferry inn coijcdg wkouv...

[3219 rows x 2 columns]


In [446]:
tf = TfidfVectorizer(ngram_range=(1,2), stop_words = STOP_WORDS, lowercase = True)
tf.fit_transform(data_train.original_text)
train_X = tf.transform(X_train.original_text)
test_X = tf.transform(X_test.original_text)
type(train_X)
# k = pd.DataFrame(X_train)
# k

C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


scipy.sparse.csr.csr_matrix

In [445]:
# train_X = X_train.copy()
train_X

,idx,original_text
2376,1.240000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
16,1.240000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
1765,1.240000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
2730,1.240000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
1500,1.250000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
...,...,...
3211,1.240000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
1351,1.250000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
528,1.250000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."
3213,1.240000e+18,"(0, 28024)\t0.39573253363137806\n (0, 28007..."


In [399]:
X_train.original_text = X_train.original_text.values
X_test.original_text = X_test.original_text.values



array([[1.25e+18,
        'happy amazing know s hard able today s protect vulnerable society pic twitter vanfjfqb',
        'en', 0, 'BeenXXPired', 0],
       [1.25e+18,
        'happy day mum m sorry t bring day honestly point d walk hot able soon love lots p s need goo mvxblrsczdte by',
        'en', 1, 'FestiveFeeling', 0],
       [1.25e+18,
        'happy day days work today quiet time reflect dog walk finish garden learn guitar drunk strawberry gin tonic watch lee evens place visit solate pic twitter gzxvvff',
        'en', 0, 'KrisAllenSak', -1],
       ...,
       [1.25e+18,
        'happy day woman know thanks pushing best person cqebsk', 'en',
        0, 'localcreativity', 0],
       [1.24e+18,
        'happy mother s day amazing wife love like crazy bit weird nxdbnpi',
        'en', 0, 'LoveluK77651882', 0],
       [1.25e+18, 'wishing safe happy day ferry inn coijcdg wkouvsqkt',
        'en', 0, 'andreaanderegg', -1]], dtype=object)

In [379]:
tf.get_feature_names()

['aamavsl',
 'aaubfsl',
 'abandoned',
 'abandoned know',
 'abandoned youth',
 'abblhtfr',
 'abiding',
 'abiding positively',
 'able',
 'able able',
 'able amazing',
 'able better',
 'able celebrate',
 'able choice',
 'able come',
 'able cuddle',
 'able daughter',
 'able day',
 'able enjoy',
 'able experience',
 'able find',
 'able fly',
 'able fresh',
 'able great',
 'able happy',
 'able hug',
 'able inside',
 'able join',
 'able juggle',
 'able kiss',
 'able laugh',
 'able let',
 'able like',
 'able live',
 'able look',
 'able mind',
 'able mum',
 'able normal',
 'able person',
 'able physically',
 'able pic',
 'able properly',
 'able ran',
 'able scattered',
 'able send',
 'able share',
 'able soon',
 'able sooner',
 'able spend',
 'able throw',
 'able time',
 'able today',
 'able video',
 'able visit',
 'able watch',
 'able wonderful',
 'abode',
 'abode like',
 'abort',
 'abort brought',
 'aborted',
 'aborted god',
 'abortion',
 'abortion happy',
 'abpruc',
 'abqmpsvnpus',
 'abroad'

In [381]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [447]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

forest = RandomForestClassifier(random_state = 1)

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, n_jobs = -1)
# print(X_train.original_text, y_train)
gridF.fit(train_X, y_train)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   57.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 22.9min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 40.7min
[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed: 53.8min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=1,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max

In [456]:
y_pred = gridF.predict(test_X)
print(f1_score(y_pred, y_test, average='weighted'))
print(confusion_matrix(y_test, y_pred))

0.698989898989899
[[  0 136   0]
 [  0 346   0]
 [  0 162   0]]


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [484]:
yy_out = tf.transform(data_test.original_text)
yy_out
yy_output = gridF.predict(yy_out)
yy_output

print([i for i in yy_output if i != 0])

[]


In [452]:
rf = RandomForestClassifier()
rf.fit(train_X, y_train)
RandomForestClassifier(bootstrap=True, class_weight=None, criterion="gini", max_depth=None, max_features='auto', max_leaf_nodes=None,min_impurity_split=1e-07, min_samples_leaf=1,min_samples_split=2, min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1, oob_score=False, random_state=None, verbose=0, warm_start=False)
y_pred = rf.predict(data_test.original_text)
print(f1_score(y_pred, y_test, average='weighted'))
confusion_matrix(y_test, y_pred)

C:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.6299022920833572


array([[  5, 122,   9],
       [ 18, 320,   8],
       [ 14, 140,   8]], dtype=int64)

In [486]:
yy_output = rf.predict(yy_out)
len(yy_output)


1387

In [490]:
a = [[idx, senti] for idx, senti in zip(data_test.id, yy_output)]
a
output = pd.DataFrame(a, columns=['id', 'sentiment_class'] )
output.to_csv('submission.csv', index = False)

In [494]:
data_test.id[0]

1.25e+18

In [498]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform
from sklearn.svm import SVC

In [496]:
param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}

In [512]:
rnd_search_cv = RandomizedSearchCV(1000, param_distributions)
# rnd_search_cv.best_estimator_


In [509]:
svm_clf = SVC(gamma="scale")
svm_clf.fit(train_X.toarray(), y_train) # We use an SVC with an RBF kernel


ValueError: could not convert string to float: 'happy mothering wonderful life living away mum t day unusual dad secret instruction othersday pic twitter vptyshcda'

In [511]:
y_pred = svm_clf.predict(test_X.toarray())
print(f1_score(y_pred, y_test, average='weighted'))
print(confusion_matrix(y_pred, y_test))

0.6965910732341472
[[  0   1   0]
 [136 345 162]
 [  0   0   0]]


C:\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
